### 作業1：
#### 練習將 DHT11 接於 GPIO 27 接腳，並且更改軟體的接腳設定值，重新執行範例程式，驗證在不同的接腳上安裝 DHT11，程式一樣可以正確地讀出數值

In [ ]:
# Ans 1：
$ sudo apt install python3-pip  #安裝 pip3 模組
$ sudo pip3 install Adafruit_DHT # 安裝 Adafruit_DHT 模組

# 在 python code，import Adafruit_DHT 後，會出現以下error：

![DHT_import_error](img/DHT_import_error.png)

In [ ]:
# Ans 2：
出現 error 的原因為：Adafruit_DHT 不能辨識 Raspberry Pi 4b CPU，
解決方式：
(1)	cd /usr/local/lib/python3.7/dist-packages/Adafruit_DHT
(2)	sudo vim platform_detect.py
(3) 在最後一個else: 前加入以下的程式碼：
elif match.group(1) == 'BCM2711': 
# Pi 4b 
return 3

![DHT_modified](img/DHT_modified.png)

In [ ]:
# Ans 3：
Python code：使用 DHT11 sensor
# 匯入 Adafruit_DHT 模組
import Adafruit_DHT
DHT_SENSOR = Adafruit_DHT.DHT11 # 感測器為 DHT11
DHT_PIN = 27 # 設定 GPIO 27 為讀取 data 的接腳
while True :
    humi, temp = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
    if (humi is not None) and (temp is not None) :
        print('Temp={:.2f}_C Humidity={:.2f}_%'.format(temp, humi))
    else :
        print('Fail to retrieve data from humidity_temp sensor')

![DHT11_reader1](img/DHT11_reader1.png)

### Ans 4：
- 更改軟體的接腳設定值，重新執行範例程式，驗證在不同的接腳上安裝 DHT11，程式一樣可以正確地讀出數值
- 使用 DHT11 讀出的數值，比使用 DHT22 會得到比較大的誤差

![DHT11_reader2](img/DHT11_reader2.png)

### 作業2：
#### 觀察 RPi.GPIO 的程式碼，與 GPIOZero 程式碼對於繼電器控制上寫法的不同，如果我們要設定 GPIO 13, 19 二個接腳控制一個二路繼電器，練習實作一個 GPIOZero 二路繼電器的控制程式

In [ ]:
# Ans 1：
# 匯入RPi.GPIO套件
import RPi.GPIO as GPIO
import time

# 使用rpi板子的GPIO編號
GPIO.setmode(GPIO.BCM)

# 將 GPIO_13、GPIO_19 設為輸出模式
relay_pins = [13, 19]
GPIO.setup(relay_pins, GPIO.OUT) # default output=LOW
GPIO.output(relay_pins, 1)

# while loop迴圈，讓relay重複動作
try :
    while True ：
    for pin in relay_pins :
        GPIO.output(pin, 0)
        time.sleep(3)
    for pin in relay_pins :
        GPIO.output(pin, 1)
        time.sleep(3)
except KeyboardInterrupt :
    pass
# 清除使用的 GPIO 狀態
GPIO.cleanup()

In [ ]:
# Ans 2：
# 匯入gpiozero套件套件
import gpiozero
import time, sys

# 將 GPIO_13、GPIO_19 設為 OutputDevice
# 初始狀態為 relay 不導通
print('initail relay 不導通')
relay_pin1 = 13
relay_pin2 = 19
relay1 = gpiozero.OutputDevice(relay_pin1, active_high=True, initial_value=False)
relay2 = gpiozero.OutputDevice(relay_pin2, active_high=True, initial_value=False)

relays = [relay1, relay2]

# while loop迴圈，讓relay重複動作
print('relay 先 on， 再 off')
try :
    while True :
        for relay_ in relays :
            relay_.on()
            time.sleep(3)
        for relay_ in relays :
            relay_.off()
            time.sleep(3)
except KeyboardInterrupt :
    print('程式停止')
    sys.exit() 

### 作業3：
#### 將作業 1 與作業 2 結合，設定程式在溫度 10 度以上到 20 度之間，控制 GPIO19，溫度 20 度到 30 度之間，控制 GPIO13，達成在不同的溫度區間時，控制不同的繼電器的需求

In [ ]:
# 匯入gpiozero套件套件
import gpiozero
import time, sys

# 匯入 Adafruit_DHT 模組
import Adafruit_DHT

DHT_SENSOR = Adafruit_DHT.DHT11 # 感測器為 DHT11
DHT_PIN = 27 # 設定 GPIO 27 為讀取 data 的接腳

# 將 GPIO_13、GPIO_19 設為 OutputDevice
print('initail relay 不導通')
relay_pin1 = 13
relay_pin2 = 19
relay1 = gpiozero.OutputDevice(relay_pin1, active_high=True, initial_value=False)
relay2 = gpiozero.OutputDevice(relay_pin2, active_high=True, initial_value=False)

relays = [relay1, relay2]

# 定義導通的 relay
def set_Relay_on(relay_n) :
    for relay_ in relays :
        if relay_n == relay_ :
            relay_.on()
        else :
            relay_.off()

# while loop迴圈，讓relay重複動作
print('溫度 10 度以上到 20 度之間，控制 GPIO19，溫度 20 度到 30 度之間，控制 GPIO13')
try :
    while True :
        humi, temp = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
        if (humi is not None) and (temp is not None) :
            temp -= 10
            print('Temp={:.2f}_C Humidity={:.2f}_%'.format(temp, humi))
            if (temp >= 10) and (temp < 20) :
                set_Relay_on(relay2)
            elif (temp >= 20) and (temp < 30) :
                set_Relay_on(relay1) 
            time.sleep(3)
        else :
            print('Fail to retrieve data from humidity_temp sensor')
except KeyboardInterrupt :
    print('程式停止')
    sys.exit()

![DHT11_controll_GPIO](img/DHT11_controll_GPIO.png)